In [49]:
from torch import nn
import torch
from einops import rearrange
from torch_geometric.datasets import Planetoid

In [50]:
def create_adjacency_matrix(edge_index, num_nodes):
    adjacency_matrix = torch.zeros((num_nodes, num_nodes), dtype=torch.float32)

    for i in range(edge_index.size(1)):
        source = edge_index[0, i]
        target = edge_index[1, i]
        adjacency_matrix[source, target] = 1

    return adjacency_matrix


def create_degree_matrix(adjacency_matrix):
    # 各行の要素の合計を計算し、それを逆数にする
    degree_vector = torch.sum(adjacency_matrix, dim=1)
    # 逆数を計算し、ゼロ除算を防ぐためにepsを加算
    inv_degree_vector = 1.0 / (degree_vector + torch.finfo(torch.float32).eps)
    # 対角行列として設定
    degree_matrix = torch.diag(inv_degree_vector)

    return degree_matrix


def create_dad(edge_index, num_nodes):
    A = create_adjacency_matrix(edge_index, num_nodes)
    D = create_degree_matrix(A)
    return D @ A @ D

In [51]:
class GraphConv(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConv, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        # 畳み込み層 (1x1 のカーネルを持つ)
        self.conv = nn.Conv2d(in_features, out_features, kernel_size=1)

    def forward(self, input, adj):
        input = rearrange(input, "V C -> C 1 V")

        # 畳み込みの適用
        XW = self.conv(input)
        XW = XW.squeeze(1)

        DADXW = torch.einsum("CV,VW->CW", XW, adj)

        # 形状を (V, C) に変換
        DADXW = rearrange(DADXW, "C V -> V C")
        return DADXW

In [52]:
# データセットを読み込む
dataset = Planetoid(root="./Cora", name="Cora")
print(
    f"""
    グラフ構造の数: {len(dataset)}
    クラス数: {dataset.num_classes}
    特徴量の次元数: {dataset.num_node_features}
    ノード数: {dataset[0].num_nodes}
    エッジ数: {dataset[0].num_edges}
"""
)


    グラフ構造の数: 1
    クラス数: 7
    特徴量の次元数: 1433
    ノード数: 2708
    エッジ数: 10556



In [53]:
data = dataset[0]

# データのマスク
train_mask = data.train_mask
val_mask = data.val_mask
test_mask = data.test_mask

In [54]:
# ノードの数
num_nodes = data.num_nodes
# 特徴量の次元数
in_channels = dataset.num_node_features
#
out_channels = dataset.num_classes
# 特徴量行列
X = dataset[0].x
# ラベル
y = dataset[0].y

# DAD行列
DAD = create_dad(dataset[0].edge_index, num_nodes)

In [55]:
model = GraphConv(in_channels, out_channels)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    out = model(X, DAD)
    loss = criterion(out[train_mask], y[train_mask])
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        # トレーニングセットの精度計算
        _, pred_train = model(X, DAD).max(dim=1)
        train_correct = int(pred_train[train_mask].eq(y[train_mask]).sum().item())
        train_acc = train_correct / int(train_mask.sum())

        # 検証セットの精度計算
        _, pred_val = model(X, DAD).max(dim=1)
        val_correct = int(pred_val[val_mask].eq(y[val_mask]).sum().item())
        val_acc = val_correct / int(val_mask.sum())

In [56]:
model.eval()
_, pred = model(X, DAD).max(dim=1)
correct = int(pred[test_mask].eq(y[test_mask]).sum().item())
test_acc = correct / int(test_mask.sum())
print(f"Test Accuracy: {test_acc:.4f}")


Test Accuracy: 0.6400
